#### Aspect-Based Sentiment Analysis (ABSA)

Aspect-Based Sentiment Analysis (ABSA) is a fine-grained form of sentiment analysis that identifies the sentiment expressed toward specific aspects or features of an entity. For example, in the review:

"The battery life of this phone is amazing, but the camera quality is disappointing."

Aspects: "battery life" and "camera quality"

Sentiments: "battery life" → Positive, "camera quality" → Negative


This approach is useful when understanding customer opinions about particular features of a product or service rather than the overall sentiment.

Steps for ABSA:

1. Aspect Extraction: Identify key aspects (features or topics) from the text.
2. Sentiment Classification: Assign sentiment (positive, negative, neutral) to each aspect.

In [11]:
import re
import json
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_groq import ChatGroq

# Initialize the Groq API with your key
groq_api_key = "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"
llm = ChatGroq(
    groq_api_key=groq_api_key,
    model="gemma-7b-it"
)

# Function to perform Aspect-Based Sentiment Analysis
def absa_analysis(review_text, language="English"):
    # Define the ABSA prompt template
    template = f"""
    Perform Aspect-Based Sentiment Analysis (ABSA) on the following review in the '{language}' language.

    For each aspect mentioned in the review, identify:
    1. The aspect (feature) being discussed.
    2. The sentiment for the aspect: "positive", "negative", or "neutral".
    
    Output the result as a JSON array. Each item in the array should include:
    - "aspect": The name of the aspect.
    - "sentiment": The sentiment for the aspect.

    Example:
    Input review: "The battery life is amazing, but the camera quality is disappointing."
    Output JSON:
    [
        {{"aspect": "battery life", "sentiment": "positive"}},
        {{"aspect": "camera quality", "sentiment": "negative"}}
    ]
    
    Review: '''{review_text}'''
    """

    # Prepare the prompt
    prompt = PromptTemplate(template=template, input_variables=["review_text", "language"])
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    
    # Run the chain with the review text and language
    response = llm_chain.run({"review_text": review_text, "language": language})

    # Debug: Print the raw response from the model
    print(f"Raw Model Response: {response}")
    
    # Extract JSON array from response using regex
    try:
        json_match = re.search(r'\[.*?\]', response, re.DOTALL)  # Matches JSON arrays
        if not json_match:
            raise ValueError("No valid JSON array found in the response.")
        
        response_json = json_match.group(0)
        aspects = json.loads(response_json)
        
        # Validate structure of the parsed JSON
        if not isinstance(aspects, list):
            raise ValueError("Parsed JSON is not a valid array.")
        for aspect in aspects:
            if not isinstance(aspect, dict) or "aspect" not in aspect or "sentiment" not in aspect:
                raise ValueError(f"Invalid JSON item structure: {aspect}")
    except (json.JSONDecodeError, ValueError) as e:
        print(f"Error processing response: {e}")
        aspects = []
    
    return aspects

# Sample Dataset for ABSA
dataset = [
    {"review": "The battery life of this phone is amazing, but the camera quality is disappointing.", "language": "English"},
    {"review": "El producto es bueno, pero el precio es alto.", "language": "Spanish"},
    {"review": "I love the display quality, but the speakers are not loud enough.", "language": "English"},
    {"review": "El servicio al cliente fue excelente, pero la entrega fue muy lenta.", "language": "Spanish"},
]

# Perform ABSA on each review
for data in dataset:
    review = data["review"]
    language = data["language"]
    aspects = absa_analysis(review, language)
    print(f"Review: {review}")
    print("Aspects and Sentiments:")
    print(json.dumps(aspects, indent=2))
    print("-" * 50)

ValueError: Missing some input keys: {'"aspect"'}